# Getting Started with Miyagi's Retrieval Augmented Generation (RaG) Workflow using Azure Cognitive Search (ACS) and Semantic Kernel

To quickly get started, follow these steps:

1. Ensure the [Polyglot notebooks extension](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode) is installed.
2. [Create a new Azure OpenAI service (or use an existing OpenAI service)](https://learn.microsoft.com/en-us/azure/ai-services/openai/chatgpt-quickstart?tabs=command-line&pivots=programming-language-studio#prerequisites).
3. [Deploy the `gpt-35-turbo` and `text-embeddings-ada-002` models](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models#working-with-models).
4. [Create an Azure Cognitive Search instance and enable the Semantic Search capability](https://learn.microsoft.com/en-us/azure/search/semantic-search-overview#enable-semantic-search).
5. Copy the `.env.example` file from the `rag` folder to `dotnet/.env` and paste the corresponding values from the resources you provisioned in the earlier steps.
6. Click on `Run All`.

> You will need an [.Net 7 SDK](https://dotnet.microsoft.com/en-us/download) and [Polyglot](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode) to get started with this notebook using .Net Interactive

## Overview
In this notebook, we will show you how to quickly get started with Miyagi's Retrieval Augmented Generation (RaG) workflow using Semantic Kernel, Azure Cognitive Search (ACS) and Cosmos. Through this use case, involving stock recommendations, you will understand the basics of RaG and the capabilities of Semantic Kernel, Azure Cognitive Search and Cosmos to adapt to your specific use case.

## Load settings from .env file

In [ ]:
#r "nuget: dotenv.net"
dotenv.net.DotEnv.Load();
var env = dotenv.net.DotEnv.Read();

## Prepare the kernel with memory using Azure Cognitive Search

We first add the nuget packages (dependecies) required to instantiate the kernel and ancillary dependencies. 

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.7.1"
#r "nuget: Microsoft.SemanticKernel.Connectors.AzureAISearch, 1.7.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.OpenApi, 1.7.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.Memory, 1.7.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.7.1-alpha"
#r "nuget: Microsoft.SemanticKernel.PromptTemplates.Handlebars, 1.7.1"
#r "nuget: Microsoft.SemanticKernel.Yaml, 1.7.1"
#r "nuget: Azure.Identity"
#r "nuget: System.IdentityModel.Tokens.Jwt, 7.5.1"
#r "nuget: Microsoft.Extensions.Logging"
#r "nuget: Microsoft.Extensions.Logging.Console"

## Importance of Authentication Using DefaultAzureCredential

Authentication is a critical aspect of security in cloud applications, ensuring that only authorized users and services can access your Azure resources. `DefaultAzureCredential` simplifies the authentication process in your .NET applications by automatically selecting the best available authentication method based on the environment your application is running in. This includes environments like local development machines, Azure-hosted applications, and CI/CD pipelines.

### How DefaultAzureCredential Works
`DefaultAzureCredential` attempts to authenticate using several methods in order. This includes:
1. Environment credentials (using environment variables)
2. Managed Identity (ideal for automated tasks in Azure, such as in Azure Functions or App Services)
3. Visual Studio credentials (helpful during development)
4. Visual Studio Code credentials
5. Azure CLI credentials (convenient for local debugging)
6. Interactive browser credential as a fallback

![Azure Identity](https://raw.githubusercontent.com/Azure/azure-sdk-for-net/main/sdk/identity/Azure.Identity/images/mermaidjs/DefaultAzureCredentialAuthFlow.svg)

### Troubleshooting Authentication Issues
If `DefaultAzureCredential` fails to authenticate, use the below code cell to troubelshoot with a detailed log setup that attempts to acquire a token using `DefaultAzureCredential`.

#### Optional: Specifying Client ID and Secret
In scenarios where `DefaultAzureCredential` does not meet your specific needs, such as when needing explicit control over the identity used for testing, you can configure a Service Principal directly using the following environment variables:
- `AZURE_CLIENT_ID`
- `AZURE_CLIENT_SECRET`
- `AZURE_TENANT_ID`

Ensure that the Service Principal has the appropriate roles assigned for accessing the Azure OpenAI Service and Azure Cognitive Search. Typically, this involves configuring role assignments in the Azure portal or using Azure CLI.

### Steps to Use Service Principal with Environment Variables
1. Create a Service Principal using Azure CLI or through the Azure portal.
2. Assign the necessary roles to the Service Principal for the resources it needs to access (for this notebook, that would be Azure OpenAI service and AI Search).
3. Set the environment variables on the machine or in the application settings where your code will run.
4. Use `DefaultAzureCredential`, which will now pick up the environment variables.

Remember, using a Service Principal requires keeping your credentials secure. Avoid hard-coding credentials in your source code and use secrets management tools provided by Azure or other third parties.



In [ ]:
using Azure.Identity;
using Microsoft.Extensions.Logging;
using System.IdentityModel.Tokens.Jwt;
using System.Linq;

var loggerFactory = LoggerFactory.Create(builder =>
{
    builder
        .AddConsole()
        .SetMinimumLevel(LogLevel.Debug);
});

ILogger logger = loggerFactory.CreateLogger("DefaultLogger");

var options = new DefaultAzureCredentialOptions
{
    Diagnostics =
    {
        IsLoggingContentEnabled = false,
        IsLoggingEnabled = true,
        IsDistributedTracingEnabled = true,
        IsTelemetryEnabled = true
    }
};

var credential = new DefaultAzureCredential(options);

try
{
    var tokenRequestContext = new Azure.Core.TokenRequestContext(new[] { "https://management.azure.com/.default" });
    var tokenResult = credential.GetToken(tokenRequestContext);
    var handler = new JwtSecurityTokenHandler();
    var jwtToken = handler.ReadJwtToken(tokenResult.Token);

    var name = jwtToken.Claims.FirstOrDefault(c => c.Type == "name")?.Value;
    var unique_name = jwtToken.Claims.FirstOrDefault(c => c.Type == "unique_name")?.Value;
    var scopes = jwtToken.Claims.FirstOrDefault(c => c.Type == "scp")?.Value;

    logger.LogInformation($"Token principal: {name}");
    logger.LogInformation($"Token Unique Name: {unique_name}");
    logger.LogInformation($"Token scopes: {scopes}");
}
catch (Exception ex)
{
    logger.LogError($"Error retrieving token: {ex.Message}");
}


With dependencies installed and configured, we create an instance of the kernel using the built-in `Kernel.Builder` class

In [4]:
using Microsoft.SemanticKernel;
using Azure.Identity;

var builder = Kernel.CreateBuilder();
        builder.Services.AddAzureOpenAIChatCompletion(
            deploymentName: env["AZURE_OPENAI_CHAT_MODEL"],
            endpoint: env["AZURE_OPENAI_ENDPOINT"],
            // credentials: new DefaultAzureCredential() // Use this for token based authentication (recommended)
            apiKey: env["AZURE_OPENAI_API_KEY"] // Use this for API key based authentication (not recommended for production use)
        );

var kernel = builder.Build();

To make the kernel stateful, we will use Memory Plugin with a Azure Cognitive Search and Embedding backend. 

In [5]:
using Microsoft.SemanticKernel.Connectors.AzureAISearch;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Plugins.Core;
  
#pragma warning disable SKEXP0003
#pragma warning disable SKEXP0011
#pragma warning disable SKEXP0021
#pragma warning disable SKEXP0001
#pragma warning disable SKEXP0010
#pragma warning disable SKEXP0020

IMemoryStore store = new AzureAISearchMemoryStore(
    env["AZURE_COGNITIVE_SEARCH_ENDPOINT"],
    // credentials: new DefaultAzureCredential() // Use this for token based authentication (recommended)
    apiKey: env["AZURE_COGNITIVE_SEARCH_API_KEY"] // Use this for API key based authentication (not recommended for production use)
    );
            
// Create an embedding generator to use for semantic memory.
var embeddingGenerator = new AzureOpenAITextEmbeddingGenerationService(
    deploymentName: env["AZURE_OPENAI_EMBEDDING_MODEL"],
    endpoint: env["AZURE_OPENAI_ENDPOINT"],
    // credential: new DefaultAzureCredential(), // Use this for token based authentication (recommended)
    apiKey: env["AZURE_OPENAI_API_KEY"], // Use this for API key based authentication (not recommended for production use)
    modelId: env["AZURE_OPENAI_EMBEDDING_MODEL"]
);

SemanticTextMemory textMemory = new(store, embeddingGenerator);

## Vectorize and persist embeddings in Azure Cognitive Search with Semantic Kernel

Embeddings are a type of data transformation that convert text into high-dimensional vectors that capture semantic relationships between words or text chunks. This vectorized form allows machines to process text in a manner akin to numerical data, enabling operations like similarity comparison which is foundational for vector databases to retrieve meaningful context for LLMs. Embeddings are essential to generate and retrieve relevant content in a RaG workflow because:

1. They normalize your data in a common mathematical representation, making it machine-friendly
1. They compress data, retaining essential representations while reducing dimensionality
1. They preserve relationships within your data
1. They are representationally dense, meaning they are more efficient to store and process


> See a visual representation of the embeddings in 2D space below (from Cohere):

![Semantic Search](https://txt.cohere.com/content/images/2023/01/Vis-7-2.jpg)

Now, let’s delve into the process of creating and utilizing embeddings in the context of ACS.

In this scenario, we have a snippet of proprietary text data—extracted from a book—which needs to be processed and indexed for semantic search. The steps involve:

1. Chunking the Text: Breaking down the text into manageable pieces, referred to as 'chunks'.
1. Generating Embeddings: Transforming these chunks into high-dimensional vectors using an Azure Text Embedding Generation Service.
1. Persisting Embeddings to ACS: Storing these embeddings in Azure Cognitive Search to facilitate semantic search.

The provided code snippet orchestrates these steps. It reads the text data from a local file, chunks it, generates embeddings for each chunk, and then persists these embeddings to ACS. The specific operations are abstracted for simplicity using methods like SaveInformationAsync, which is part of a hypothetical kernel object in this code. The end result is a collection of indexed embeddings in ACS that significantly enhance search quality by moving beyond mere keyword matching to understanding semantic relationships.

In [6]:
using Microsoft.SemanticKernel.Text;
using System.IO;

#pragma warning disable SKEXP0055
#pragma warning disable SKEXP0050
var dataset = "intelligent-investor.txt";
var recommendationServicePath = "../../../../services/recommendation-service/dotnet";
const int MaxTokensPerParagraph = 300;
const int MaxTokensPerLine = 100;

// Load text data from a local file
var filePath = Path.Combine(recommendationServicePath, "resources", "sample-datasets", dataset);
var streamReader = new StreamReader(filePath);
var text = await streamReader.ReadToEndAsync();

// Define a memory collection name for indexing
var memoryCollectionName = "miyagi-investment-thesis";

// Chunk the text into lines and paragraphs for manageable processing
var lines = TextChunker.SplitPlainTextLines(text, MaxTokensPerLine);
var chunks = TextChunker.SplitPlainTextParagraphs(lines, MaxTokensPerParagraph);

// Loop through each chunk to generate embeddings and persist to ACS
for (var i = 0; i < chunks.Count; i++)
{
    var chunk = chunks[i];
    // Generate an identifier for each chunk
    await textMemory.SaveInformationAsync(
        memoryCollectionName,
        chunk,
        $"{dataset}-{i}",
        $"Dataset: {dataset} Chunk: {i}",
        i.ToString());
}

// Output the result to console
System.Console.WriteLine($"Saved {chunks.Count} chunks to memory collection {memoryCollectionName}");


Saved 24 chunks to memory collection miyagi-investment-thesis


## Search and retrieve documents using Semantic Kernel

To search and retrieve documents, we will use the `SearchAsync` method of Semantic memory that encapsulates retrieval abstractions of different memory providers. This method takes in a query and returns a list of documents that are relevant to the query.

In [7]:
 var query = "Ben Graham's investment philosophy";
 
 await foreach (var result in textMemory.SearchAsync(
            collection: memoryCollectionName,
            query: query,
            limit: 2,
            minRelevanceScore: 0.79,
            withEmbeddings: true))
        {
            Console.WriteLine("   " + result.Metadata.Text);
            Console.WriteLine("   Relevance: " + result.Relevance + "\n");
        }

   Projection is dangerous because the future is uncertain, and inflation, economic recessions, pandemics and geopolitical upheavals often arrive without warning.
Graham argues that it is a fool's errand for an ordinary investor to attempt making money by timing the market.
Graham advocates for a protection-based approach that does not try to time the market.
Value investors should identify and invest in large, conservatively financed companies whose present value (as estimated by tangible assets) is substantially below their current stock prices.
   Relevance: 0.8646199382230626

   Similarly, a defensive investor who has made a value-based investment based on sound business fundamentals will ignore the stock market valuation apart from taking advantage of its swings. Graham goes so far as to say that the single biggest reason investors fail is that they pay too much attention to what the stock market is currently doing.
Intelligent investors should be comfortable holding their stocks

## Grounding with Native Functions and SK's Memory "recall"

> Note that this prompt template (semantic function) is located under services/reccommendation-service/dotnet/plugins/AdvisorPlugin

In [8]:
using System.IO;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.PromptTemplates.Handlebars;
using Microsoft.SemanticKernel.Plugins.Memory;

#pragma warning disable SKEXP0052
#pragma warning disable SKEXP0001
#pragma warning disable SKEXP0050


var pluginFolder = $"{recommendationServicePath}/Resources/Prompts";

// Construct the full file path
var filePath = Path.Combine(pluginFolder, "InvestmentAdvise.prompt.yaml");

KernelFunction advisorPlugin;

// Use StreamReader to read the file
using (StreamReader reader = new StreamReader(filePath))
{
    // Create the function from the YAML file using Handlebars template
    advisorPlugin = kernel.CreateFunctionFromPromptYaml(
        reader.ReadToEnd(),
        promptTemplateFactory: new HandlebarsPromptTemplateFactory()
    );
}


// recall is from the TextMemoryPlugin, which has a recall method that can be invoked from Semantic Function to perform retrieval
KernelPlugin memoryPlugin = kernel.ImportPluginFromObject(new TextMemoryPlugin(textMemory));

### Set Kernel Arguments (context variables)

In [ ]:
using System.Text.Json;
using Microsoft.SemanticKernel.ChatCompletion;

#pragma warning disable SKEXP0052
#pragma warning disable SKEXP0001
#pragma warning disable SKEXP0050

// Create few-shot examples
List<ChatHistory> fewShotExamples = [
    [
        new ChatMessageContent(AuthorRole.User, @"{""stocks"":[{""symbol"":""MSFT"",""allocation"":0.6},{""symbol"":""ACN"",""allocation"":0.4}]}"),
        new ChatMessageContent(AuthorRole.Assistant, @"{""portfolio"":[{""symbol"":""MSFT"",""gptRecommendation"":""Booyah! Hold on, steady growth! Diversify, though!""},{""symbol"":""ACN"",""gptRecommendation"":""Buy! Services will see a boom!""}]}")
    ]
];

/*
    * Resolve issue with Handlebars ProcessPositionalArguments issue with parameter parsing for TextMemoryPlugin-recall
    * In prompt template for handlebars, the following is not working:
    * {{TextMemoryPlugin-recall semanticQuery, miyagi-embeddings, 0.8, 3}}
    *
    * Meanwhile, use this alternative approach
*/
var memories = await kernel.InvokeAsync(memoryPlugin["Recall"], new()
{
    [TextMemoryPlugin.InputParam] = $"Investment advise for aggressive risk level",
    [TextMemoryPlugin.CollectionParam] = "miyagi-investment-thesis",
    [TextMemoryPlugin.LimitParam] = "2",
    [TextMemoryPlugin.RelevanceParam] = "0.8",
});

Console.WriteLine($"Memories: {memories.GetValue<string>()}");
Console.WriteLine();

// Set harcoded parameters for the AdvisorPlugin
var stocks = new[] {
    new {symbol = "MSFT", allocation = 0.3},
    new {symbol = "ACN", allocation = 0.1},
    new {symbol = "JPM", allocation = 0.3},
    new {symbol = "PEP", allocation = 0.3}
};

var arguments = new KernelArguments
{
    ["userId"] = "50",
    ["stocks"] = JsonSerializer.Serialize(stocks),
    ["risk"] = "aggressive",
    ["fewShotExamples"] = fewShotExamples,
    ["voice"] = "Jim Cramer",
    ["memories"] = memories.GetValue<string>()
};
arguments

### Create native function

In [10]:
// Copyright (c) Microsoft. All rights reserved.

using System.ComponentModel;
using Microsoft.SemanticKernel;


/// <summary>
///     UserProfilePlugin shows a native skill example to look user info given userId.
/// </summary>
/// <example>
///     Usage: kernel.ImportSkill("UserProfilePlugin", new UserProfilePlugin());
///     Examples:
///     SKContext["userId"] = "000"
///     {{UserProfilePlugin.GetUserAge $userId }} => {userProfile}
/// </example>
public class UserProfilePlugin
{
    /// <summary>
    ///     Name of the context variable used for UserId.
    /// </summary>
    public const string UserId = "UserId";

    private const string DefaultUserId = "40";
    private const int DefaultAnnualHouseholdIncome = 150000;
    private const int Normalize = 81;

    /// <summary>
    ///     Lookup User's age for a given UserId.
    /// </summary>
    /// <example>
    ///     SKContext[UserProfilePlugin.UserId] = "000"
    /// </example>
    [KernelFunction, Description("Given a userId, get user age")]
    public string GetUserAge(
        [Description("Unique identifier of a user")]
        string userId)
    {
        // userId = context.Variables.ContainsKey(UserId) ? context[UserId] : DefaultUserId;
        userId = string.IsNullOrEmpty(userId) ? DefaultUserId : userId;
        
        int age;

        if (int.TryParse(userId, out var parsedUserId))
            age = parsedUserId > 100 ? parsedUserId % Normalize : parsedUserId;
        else
            age = int.Parse(DefaultUserId);

        // invoke a service to get the age of the user, given the userId
        return age.ToString();
    }

    /// <summary>
    ///     Lookup User's annual income given UserId.
    /// </summary>
    /// <example>
    ///     SKContext[UserProfilePlugin.UserId] = "000"
    /// </example>
    [KernelFunction, Description("Given a userId, get user annual household income")]
    public string GetAnnualHouseholdIncome(
        [Description("Unique identifier of a user")]
        string userId)
    {
        
        // userId = context.Variables.ContainsKey(UserId) ? context[UserId] : DefaultUserId;
        userId = string.IsNullOrEmpty(userId) ? DefaultUserId : userId;

        var random = new Random();
        var randomMultiplier = random.Next(1000, 8000);

        // invoke a service to get the annual household income of the user, given the userId
        var annualHouseholdIncome = int.TryParse(userId, out var parsedUserId)
            ? parsedUserId * randomMultiplier
            : DefaultAnnualHouseholdIncome;

        return annualHouseholdIncome.ToString();
    }
}

In [11]:
// import the UserProfilePlugin
kernel.Plugins.AddFromType<UserProfilePlugin>();

### Invoke the LLM

In [12]:
var result = await kernel.InvokeAsync(
            advisorPlugin,
            arguments
        );
result.GetValue<string>()

{"portfolio":[
  {"symbol":"MSFT","gptRecommendation":"Tech's a battlefield, but Microsoft's a warrior!"},
  {"symbol":"ACN","gptRecommendation":"Accenture's hot, but don't forget to spread your bets!"},
  {"symbol":"JPM","gptRecommendation":"Bank on JPMorgan, it's a financial fortress!"},
  {"symbol":"PEP","gptRecommendation":"Pepsi's fizzing with potential, keep sipping those dividends!"}
]}

![RaG Workflow](../../../../assets/images/sk-memory-orchestration.png)